<a href="https://colab.research.google.com/github/fcotic22/CASE26_projekt/blob/main/CASE_projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalacija potrebnih modula sa pip alatom, otkomentirati kod prvog pokretanja

In [3]:
#!pip install -q datasets
#!pip install -q torch
#!pip install -q unsloth

### Preuzimanje dataseta sa hugging face-a

### Odabir i učitavanje modela

In [4]:
import unsloth
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

==((====))==  Unsloth 2026.2.1: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### Učitavanje dataseta i tokeniziranje podataka za davanje modelu za treniranje

In [5]:

prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}
### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def promptFormatter(prompts):
    instructions = prompts["instruction"]
    outputs = prompts["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("fcotic/solarnaEnergija", split="train")
dataset = dataset.map(promptFormatter, batched = True,)
print(dataset[0])


{'instruction': 'Objasni što su fotonaponski sustavi i koja im je osnovna svrha.', 'output': 'Fotonaponski sustavi su energetski sustavi koji pretvaraju Sunčevo zračenje izravno u električnu energiju pomoću fotonaponskih ćelija. Njihova osnovna svrha je proizvodnja električne energije za napajanje istosmjernih ili izmjeničnih trošila, bilo samostalno ili u kombinaciji s elektroenergetskom mrežom. Ovi sustavi omogućuju korištenje obnovljivih izvora energije, smanjuju emisije CO₂ i doprinose energetskoj neovisnosti korisnika.', 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n### Instruction:\nObjasni što su fotonaponski sustavi i koja im je osnovna svrha.\n### Response:\nFotonaponski sustavi su energetski sustavi koji pretvaraju Sunčevo zračenje izravno u električnu energiju pomoću fotonaponskih ćelija. Njihova osnovna svrha je proizvodnja električne energije za napajan

### Konfiguracija parametara modela


In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 60, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

### Fino podešavanje na pripremljenim podacima

In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 997 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


Step,Training Loss
1,1.890600
2,1.873000
3,1.662300
4,1.519100
5,1.522400
6,1.528000
7,1.176100
8,1.246400
9,1.324000
10,0.970500


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇█
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,██▇▇▄▅▅▁▃▃▂▂▄▅▃▃▃▂▂▂▃▃▂▃▂▂▃▁▂▃▃▃▄▃▃▂▃▃▂▂
train/learning_rate,▁▁▂▅▇███▇▇▇▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁
train/loss,██▇▆▆▃▄▄▂▃▂▃▃▄▃▃▃▃▂▂▂▂▂▃▂▂▃▂▃▂▂▂▁▃▄▃▂▂▂▁
total_flos,4594978264031232.0
train/epoch,0.48144
train/global_step,60
train/grad_norm,1.25629
train/learning_rate,0.0
train/loss,0.8401


### Evaluacija novog modela


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}
### Response:
{}"""

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Zašto bi ugradio solarne panele na svoju kuću, koje su prednosti toga?",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
model.save_pretrained("solarSystemsModel") # Local saving
tokenizer.save_pretrained("solarSystemsModel")

#from google.colab import files
#files.download("solarSystemsModel")
